# Modelado

Ingenieria de variables básica, modelado, y predicción.

In [1]:
# dependencies
import numpy  as np
import pandas as pd
from   sklearn.impute          import SimpleImputer
from   sklearn.preprocessing   import OneHotEncoder, StandardScaler
from   sklearn.model_selection import GridSearchCV
from   sklearn.ensemble        import RandomForestClassifier
from   sklearn.tree            import DecisionTreeClassifier
from   sklearn.metrics         import make_scorer, f1_score

In [2]:
# funciones
# fe variables numericas
def feature_engineer_numeric(dataframe, columnas_numericas):
    df = dataframe.copy()
    # imputación
    imputador_numeric   = SimpleImputer(missing_values = np.nan, strategy = 'mean').fit(df[columnas_numericas])
    df_imputed = imputador_numeric.transform(df[columnas_numericas])    
    # estandarización
    estandarizador = StandardScaler().fit(df_imputed)
    df_standarizado = estandarizador.transform(df_imputed)
    # add names
    df_nombres = pd.DataFrame(df_standarizado, columns = columnas_numericas).reset_index(drop = True)
    df_salida  = pd.concat([df_nombres, dataframe.drop(columns = columnas_numericas)], axis = 1)
    #
    return df_salida
# fe variables categoricas
def feature_engineer_categoric(dataframe, columnas_categoricas):
    df = dataframe.copy()
    # imputacion
    imputador_categoric = SimpleImputer(missing_values = None, strategy = 'most_frequent').fit(df[columnas_categoricas])
    df_imputed = imputador_categoric.transform(df[columnas_categoricas])
    # ohe
    encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False).fit(df_imputed)
    df_ohe = encoder.transform(df_imputed)
    # add names
    columnas_nombres = encoder.get_feature_names()
    df_nombres = pd.DataFrame(df_ohe, columns = columnas_nombres).reset_index(drop = True)
    # gather results
    df_salida = pd.concat([dataframe.drop(columns = columnas_categoricas), df_nombres], axis = 1)
    #
    return df_salida
# junto ambas
def feature_engineer(dataframe, columnas_numericas, columnas_categoricas):
    df_numeric   = feature_engineer_numeric(dataframe,    columnas_numericas)
    df_categoric = feature_engineer_categoric(df_numeric, columnas_categoricas)
    # salida
    return df_categoric
# preparación de la variable objetivo
def ohe_objetivo(dataframe):
    df = dataframe.copy()
    # 
    encoder = OneHotEncoder(handle_unknown = 'error', sparse = False).fit(df)
    df_ohe  = encoder.transform(df)
    # format
    columnas_nombres = encoder.get_feature_names()
    df_nombres       = pd.DataFrame(df_ohe, columns = columnas_nombres).reset_index(drop = True)
    # salida
    return df_nombres, encoder

In [3]:
# feature engineer for new examples, with any number of missing values
# numeric
def feature_engineer_numeric_new_data(dataframe_parcial, dataframe_completo, columnas_numericas):
    df_completo = dataframe_completo.copy()
    df_parcial = dataframe_parcial.copy()
    ##  transformations
    # imputacion
    imputador_numeric = SimpleImputer(missing_values = np.nan, strategy = 'mean').fit(df_completo[columnas_numericas])
    df_imputed        = imputador_numeric.transform(df_parcial[columnas_numericas])
    # estandarizado
    estandarizador  = StandardScaler().fit(df_completo[columnas_numericas])
    df_standarizado = estandarizador.transform(df_imputed)
    # add names
    df_nombres = pd.DataFrame(df_standarizado, columns = columnas_numericas).reset_index(drop = True)
    df_salida  = pd.concat([df_nombres, df_parcial.drop(columns = columnas_numericas)], axis = 1)
    # 
    return df_salida
# ingenieria de variables para las variables categoricas de los ejemplos nuevos
def feature_engineer_categoric_new_data(dataframe_parcial, dataframe_completo, columnas_categoricas):
    df_completo = dataframe_completo.copy()
    df_parcial  = dataframe_parcial.copy()
    # imputacion
    imputador_categoric = SimpleImputer(missing_values = '', strategy = 'most_frequent').fit(df_completo[columnas_categoricas])
    df_imputed = imputador_categoric.transform(df_parcial[columnas_categoricas])
    # ohe
    encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False).fit(df_completo[columnas_categoricas])
    df_ohe = encoder.transform(df_imputed)
    # add names
    columnas_nombres = encoder.get_feature_names()
    df_nombres = pd.DataFrame(df_ohe, columns = columnas_nombres).reset_index(drop = True)
    # gather results
    df_salida = pd.concat([dataframe_parcial.drop(columns = columnas_categoricas), df_nombres], axis = 1)
    #
    return df_salida
# junto ambos pasos
def feature_engineer_new_data(dataframe_parcial, dataframe_completo, columnas_numericas, columnas_categoricas):
    df_completo = dataframe_completo.copy()
    df_parcial  = dataframe_parcial.copy()
    # magic
    df_numeric   = feature_engineer_numeric_new_data(df_parcial,   df_completo, columnas_numericas)
    df_categoric = feature_engineer_categoric_new_data(df_numeric, df_completo, columnas_categoricas)
    # 
    df_salida = df_categoric
    return df_salida

In [4]:
# prediction of new samples
# function to fill empty values in user examples
def complete_df(df_example):
    # helper df
    df_empty = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, '', '', '', '']], columns = columnas_predictoras)
    # llenado
    df_salida = pd.concat([df_empty.drop(columns = df_ejemplo_raw.columns), df_ejemplo_raw], axis = 1)
    # 
    return df_salida
#  prediction function
# function to predict in human terms
def predict_phone(ejemplo, dataframe_entrenamiento, columnas_numericas, columnas_categoricas, modelo, encoder_objetivo, dataframe_completo):
    # transformación para la prediccion
    df_ejemplo_transformed = feature_engineer_new_data(ejemplo, dataframe_entrenamiento, columnas_numericas, columnas_categoricas)
    # predicción
    prediccion_cruda = modelo.predict(df_ejemplo_transformed)
    # formato más familiar
    prediction_human = encoder_objetivo.inverse_transform(prediccion_cruda)
    # agrupo todo en un dataframe
    df_salida = pd.DataFrame({'producto_nombre': prediction_human[0]}).merge(dataframe_completo, on = 'producto_nombre', how = 'inner')
    # 
    return df_salida

In [5]:
# data
path = 'https://raw.githubusercontent.com/yoselalberto/ia_proyecto_final/main/data/processed/celulares_procesados.csv'

In [6]:
# loading
df_inicio = pd.read_csv(path)

In [7]:
# variables globales
# estas columnas serán ignoradas durante el modelado
columnas_ignorar     = {'color', 'pantalla'}
# variable objetivo
columna_objetivo     = 'producto_nombre'
# columnas categoricas
columnas_categoricas = ['marca', 'procesador', 'sistema_operativo', 'tecnologia']
# columnas numericas
columnas_numericas   = ['peso', 'camara_trasera', 'camara_frontal', 'ram', 'memoria', 'precio']
# variables predictoras
columnas_predictoras = columnas_numericas + columnas_categoricas

In [8]:
# elimino columnas, duplicados, y reordeno las columnas
df = df_inicio.drop(columns = columnas_ignorar).drop_duplicates().reset_index(drop = True)[columnas_predictoras + [columna_objetivo]]

In [9]:
df

,peso,camara_trasera,camara_frontal,ram,memoria,precio,marca,procesador,sistema_operativo,tecnologia,producto_nombre
0,0.282,12,10,12,256,46799,samsung,qualcomm,android,5g,galaxy z fold2
1,0.272,12,12,4,512,33999,apple,apple,ios,4glte,iphone 11 pro max
2,0.252,12,12,4,512,31279,apple,apple,ios,4glte,iphone 11 pro
3,0.302,12,10,8,256,30599,samsung,qualcomm,android,4g,galaxy note 20 ultra
4,0.183,12,10,8,256,29699,samsung,qualcomm,android,4glte,galaxy z flip
...,...,...,...,...,...,...,...,...,...,...,...
76,0.149,13,8,2,32,2990,motorola,qualcomm,android,4glte,moto e6 plus
77,0.190,13,8,1,16,2499,motorola,qualcomm,android,4glte,moto e6 play
78,0.149,13,8,2,32,2990,motorola,mediatek,android,4glte,moto e6 plus
79,0.176,13,8,2,32,3299,huawei,mediatek,android,4glte,honor 8a


In [13]:
df.marca.unique()

array(['samsung', 'apple', 'motorola', 'xiaomi', 'huawei', 'tcl', 'nokia'],
      dtype=object)

## Train and test split
Se ocuparán todos los datos para el modelado, se reportará el cross validation error. Las buenas prácticas indican usar un conjunto no visto en el entrenamiento, una excepció es cuando los datos son pocos, justo como en este caso; el cross validation error será más optimista que su desempeño real, pero es la mejor solución en esté caso.

In [10]:
predictores = df.drop(columns = columna_objetivo)
objetivo    = df[[columna_objetivo]]

## Ingeniería de variables

Para llenar valores faltantes usaremos el promedio para las variables númericas, y la moda para las variables categoricas; también estandarizamos las variables númericas, restamos la media, y dividimos entre la desviación estandar.  
Para la variable objetivo, simplemente aplicamos el one hot encoding.

In [11]:
predictores_transformed = feature_engineer(predictores, columnas_numericas, columnas_categoricas)

In [12]:
(objetivo_transformed, encoder_objetivo) = ohe_objetivo(objetivo)

## Modelado

Al final implementaremos un RandomForest, de la documentación oficial:

<cite>A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.</cite>

In [13]:
# creo evaluador para el grid search, utilizaré el micro f1 score
scorer_f1  = make_scorer(f1_score, average = 'micro')
# valores a explorar
# parameters = {'max_depth': [2, 4, 8], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1, 2, 4], 'n_estimators': [5, 10, 20]}
parameters = {'max_depth': [2, 4, 8], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [1, 2, 4]}
# ajuste y evaluacion
# modelo = GridSearchCV(RandomForestClassifier(random_state = 59, oob_score = True, max_features = 6, class_weight = "balanced"), parameters, n_jobs = 6, scoring = scorer_f1, cv = 4)
modelo = GridSearchCV(DecisionTreeClassifier(random_state = 59, max_features = 6, class_weight = "balanced"), parameters, n_jobs = 6, scoring = scorer_f1, cv = 5)
# ajuste modelo
modelo.fit(X = predictores_transformed, y = objetivo_transformed)
# extraigo el mejor
modelo_mejor = modelo.best_estimator_
# ojeada
print(modelo.best_score_, modelo.best_params_) 

0.38014705882352945 {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 1}


## Preparación ejemplos nuevos

Aquí transformamos la entrada del usuario, manejando pósibles valores faltantes. Hacemos la predicción, y devolvemos una tabla con la recomendación

In [14]:
# example
df_ejemplo_raw = pd.DataFrame({'precio': 5000, 'sistema_operativo': 'android', 'camara_trasera': 12, 'memoria': 32, 'ram' : 4}, index = [0])
# completing with default values
df_ejemplo = complete_df(df_ejemplo_raw)
# vistazo de la entrada
df_ejemplo

,peso,camara_frontal,marca,procesador,tecnologia,precio,sistema_operativo,camara_trasera,memoria,ram
0,NaN,NaN,,,,5000,android,12,32,4


In [15]:
# prediction
predict_phone(df_ejemplo, predictores, columnas_numericas, columnas_categoricas, modelo_mejor, encoder_objetivo, df_inicio)

,producto_nombre,marca,color,peso,pantalla,camara_trasera,camara_frontal,procesador,ram,memoria,sistema_operativo,precio,tecnologia
0,galaxy a20s,samsung,azul,0.185,tft-lcd,13,8,qualcomm,4,32,android,4999,4glte
1,galaxy a20s,samsung,rojo,0.185,super amoled,13,8,qualcomm,4,32,android,4769,4glte
2,galaxy a20s,samsung,negro,0.185,super amoled,13,8,qualcomm,4,32,android,4190,4glte


### Ejemplo paso a paso



In [16]:
# prediction, paso a paso
# def predict_phone(ejemplo, dataframe_entrenamiento, columnas_numericas, columnas_categoricas, modelo, encoder_objetivo, dataframe_completo):
# transformación para la prediccion
df_ejemplo_transformed = feature_engineer_new_data(df_ejemplo, predictores, columnas_numericas, columnas_categoricas)
# predicción
prediccion_cruda = modelo_mejor.predict(df_ejemplo_transformed)
print(prediccion_cruda)
# formato más familiar
prediction_human = encoder_objetivo.inverse_transform(prediccion_cruda)
print(prediction_human)
# agrupo todo en un dataframe
# pd.DataFrame({'producto_nombre': prediction_human[0]}).merge(df_inicio, on = 'producto_nombre', how = 'inner')
# 
#     return df_salida

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[['galaxy a20s']]
